# Module 6 - GraphRAG and Agent

In the following notebook are we creating an agent on top of the Graph.

Import our usual suspects (and some more...)

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from neo4j import Query, GraphDatabase, RoutingControl, Result
from langchain.schema import HumanMessage
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.prebuilt import create_react_agent
from openai import OpenAI
from typing import List, Optional
from pydantic import BaseModel, Field, validator
import functools
from langchain_core.tools import tool
import gradio as gr
import time
from json import loads, dumps
from typing_extensions import TypedDict
from typing import Literal
from langgraph.graph import StateGraph, Graph, START, END
from langgraph.checkpoint.memory import MemorySaver

## Setup

Load env variables

In [3]:
env_file = 'ws.env'

In [4]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

Setup connection to the database with the [Python Driver](https://neo4j.com/docs/python-manual/5/).

In [5]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

Test the connection

In [6]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,1388


Test whether we got our constraints

In [7]:
schema_result_df  = driver.execute_query(
    'show indexes',
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

In [8]:
schema_result_df.head(100)

,id,name,state,populationPercent,type,entityType,labelsOrTypes,properties,indexProvider,owningConstraint,lastRead,readCount
0,6,chunk-embeddings,ONLINE,100.0,VECTOR,NODE,[Chunk],[embedding],vector-2.0,None,2025-05-16T10:47:05.495000000+00:00,128
1,7,definition-embeddings,ONLINE,100.0,VECTOR,NODE,[Definition],[embedding],vector-2.0,None,2025-05-16T10:47:05.506000000+00:00,752
2,0,index_343aff4e,ONLINE,100.0,LOOKUP,NODE,None,None,token-lookup-1.0,None,2025-05-16T10:47:42.559000000+00:00,7212
3,1,index_f7700477,ONLINE,100.0,LOOKUP,RELATIONSHIP,None,None,token-lookup-1.0,None,2025-05-16T10:45:38.133000000+00:00,146
4,2,unique_chunk,ONLINE,100.0,RANGE,NODE,[Chunk],[id],range-1.0,unique_chunk,2025-05-16T10:02:22.855000000+00:00,7026
5,4,unique_document,ONLINE,100.0,RANGE,NODE,[Document],[id],range-1.0,unique_document,2025-05-15T09:24:03.456000000+00:00,625


## Agents with GraphRAG

### Lets create a Retrieval agent

In [9]:
client = OpenAI()

In [10]:
llm = ChatOpenAI(model_name=LLM, temperature=0)

In [11]:
llm.model_name

'gpt-4o'

In [12]:
embedding_model = OpenAIEmbeddings(
    model=EMBEDDINGS_MODEL,
    openai_api_key=OPENAI_API_KEY
)

In [13]:
embedding_model.model

'text-embedding-ada-002'

### Tool 1 - Retrieve Products

Define a function that retrieves products from the Database

In [14]:
def retrieve_products() -> pd.DataFrame:
    """Retrieve the products in the database. Products are specified with name. """
    return driver.execute_query(
        """
        MATCH (p:ProductType)
        RETURN p.name as name
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.to_df(),
    )['name'].tolist()

In [15]:
retrieve_products()

['SpaarRekening',
 'DirectRekening',
 'Kortlopende Reis',
 'BeleggersRekening',
 'RaboBusiness Banking']

### Tool 2 - Map Products to Database

To map a product from some a question we need to map it to the database.

In [16]:
map_products_prompt = """
As an intelligent assistant, your primary objective is to map a product name to product names in the database.

Examples:
#####
Product: savings account. 
Database Products: ['SpaarRekening', 'DirectRekening', 'Kortlopende Reis', 'BeleggersRekening', 'RaboBusiness Banking']
Assistant: Product: SpaarRekening
#####
#####
Product: Direct Rekening. 
Database Products: ['SpaarRekening', 'DirectRekening', 'Kortlopende Reis', 'BeleggersRekening', 'RaboBusiness Banking']Assistant: Customer: Jan Blok
Assistant: Product: DirectRekening

#####
#####
Product: Reis verzekering. 
Database Products: ['SpaarRekening', 'DirectRekening', 'Kortlopende Reis', 'BeleggersRekening', 'RaboBusiness Banking']Assistant: Customer: Jan Blok
Assistant: Product: Kortlopende Reis
#####
"""

In [17]:
def map_product_to_database_products(product) -> str:
    """Map products from the user question to the actual products in the database."""

    response = client.beta.chat.completions.parse(
        model=LLM,
        temperature=0,
        messages=[
            {"role": "system", "content": map_products_prompt},
            {"role": "user", "content": "Product: " + product},
            {"role": "user", "content": "Database Products: " + str(retrieve_products())},
            
        ],
#        response_format=DefinitionList,
    )
    return response.choices[0].message.content 

In [18]:
map_product_to_database_products('savings account')

'Product: SpaarRekening'

### Tool 3 - Retrieve product documents

Define a function that retrieves a document from a specific product

In [19]:
def retrieve_document_from_product(product_name) -> pd.DataFrame:
    """Retrieve the documents of products in the database. Products are specified with their name. """
    return driver.execute_query(
        """
        MATCH (p:ProductType)<-[:RELATED_TO]-(d:Document)
        WHERE LOWER(p.name) = LOWER($product_name)
        RETURN d.file_name
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        product_name = product_name,
        result_transformer_= lambda r: r.to_df(),
    ).iloc[0]['d.file_name']

In [20]:
retrieve_document_from_product('SpaarRekening')

'Rabo SpaarRekening 2020.pdf'

### Tool 4 - GraphRAG Document Search

The following function performs GraphRAG but only on a specific document.

In [21]:
def get_context_graphrag(search_prompt, document):
    query_vector = embedding_model.embed_query(search_prompt)

    similarity_query = """ 
        CALL db.index.vector.queryNodes("chunk-embeddings", 30, $query_vector) YIELD node, score
        WITH node as chunk, score ORDER BY score DESC
        CALL (chunk) {
            MATCH (chunk)-[r:OVERLAPPING_DEFINITIONS]-(overlapping_chunk:Chunk)
            WHERE r.overlap > 3
            RETURN collect(overlapping_chunk) AS overlapping_chunks
        }
        WITH [chunk] + overlapping_chunks AS chunks
        UNWIND chunks as chunk
        MATCH (d:Document{file_name: $document})<-[:PART_OF]-(chunk)
        RETURN d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk_eng AS chunk
       """
    results_1 = driver.execute_query(
        similarity_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        query_vector=query_vector,
        document=document,
        result_transformer_= lambda r: r.to_df()
    )

    chunk_ids = list(set(results_1['chunk_id'].to_list()))

    definition_query = """    
       CALL db.index.vector.queryNodes("definition-embeddings", 5, $query_vector) YIELD node, score
            WITH node as definition, score ORDER BY score DESC
            WHERE definition.degree < 20
            WITH definition LIMIT 1
            MATCH (definition)<-[:MENTIONS]-(chunk:Chunk)
            WHERE NOT (chunk.id IN $chunk_ids)
            WITH chunk LIMIT 3
            MATCH (d:Document{file_name: $document})<-[:PART_OF]-(chunk)
            RETURN d.file_name as file_name, chunk.id as chunk_id, chunk.page as page, chunk.chunk_eng AS chunk
    """
    results_2 = driver.execute_query(
        definition_query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        chunk_ids=chunk_ids,
        document=document,
        query_vector=query_vector,
        result_transformer_= lambda r: r.to_df()
    )

    results = pd.concat([results_1,results_2]).drop_duplicates()
    results = results.to_json(orient="records")
    parsed = loads(results)
    context = dumps(parsed, indent=2)
    return context

In [22]:
def generate_prompt(search_prompt, context):
    prompt_template = """

    You are a chatbot on Rabobank product. Your goal is to help people with questions on product policies.  
    A user will come to you with questions on their policy. Their questions must be answered based on the relevant documents of the policy.
    Respond in English. 

    The question is the following: 
    {search_prompt}
    
    Always respond in the language in which the question was asked. So, do not respond in a different language.
    
    The context is the following: 
    {context}

    Please explain your answer as thorough as possbile based on the context above. Don't come up with anything yourself.
    
    Please end your message with listing your sources with file name and page number. 
    """
    prompt = PromptTemplate.from_template(prompt_template)
    
    theprompt = prompt.format_prompt(search_prompt=search_prompt, context=context)
    return theprompt

In [23]:
context = get_context_graphrag("What are the rules for shared savings account?", "Rabo SpaarRekening 2020.pdf")

In [24]:
print(context)

[
  {
    "file_name": "Rabo SpaarRekening 2020.pdf",
    "chunk_id": 79,
    "page": 10,
    "chunk": "You must comply with these.  \n5 For (the use of) an online service, the instructions and regulations included in the general terms and conditions applicable to that online service apply.  \n20. Bankruptcy, suspension of payments, statutory debt restructuring, and seizure  \n1 If you are bankrupt or in statutory debt restructuring, you may no longer use the savings account. If the court grants you a suspension of payments, the law refers to this as suspension of payments. You may then no longer use the savings account alone, but only together with your administrator.  \n2 Is the savings account held by multiple account holders and does one or more of the account holders experience one of the cases mentioned in paragraph 1? Then none of you may use the savings account anymore. We reserve the right to offset a debt that one of you owes us with the balance.  \n3 If the savings account i

In [25]:
def perform_search_in_document(document, search_prompt) -> [str, str]:
    """Peform a search in the document to search relevant text to answer a user question. The document first needs to be determined before a search should be performed."""
    context = get_context_graphrag(document, search_prompt)
    return context

In [26]:
def answer_question_in_document(question, document) -> str:
    """This function is answering a question based on a search in a document (vector search on document). Document and question both need to be provided."""
    context = perform_search_in_document(question, document)
    theprompt = generate_prompt(question, context)
    return llm(theprompt.to_messages()).content

In [27]:
result = answer_question_in_document("What are the rules for shared savings account?", "Rabo SpaarRekening 2020.pdf")

/var/folders/l0/vn55w75s41703t1w811dtf080000gp/T/ipykernel_23624/881515064.py:5: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(theprompt.to_messages()).content


In [28]:
print(result)

The rules for a shared savings account, as outlined in the provided context, are as follows:

1. **Joint Account**: If the savings account has multiple account holders, it is considered a joint account unless otherwise agreed upon as a joint-and account. For both types of accounts, the following rules apply:
   - Only one account holder needs to be informed about account matters, and it is their responsibility to inform the other account holders. This applies even if the account holders do not live at the same address (Rabo SpaarRekening 2020.pdf, page 8, chunk 68).
   - If an account holder dies, their heirs can use the account together in their place (Rabo SpaarRekening 2020.pdf, page 8, chunk 69).

2. **And/Or Account**: 
   - Account holders can use the savings account separately, even after the death of one of the account holders or if a judge places an account holder under guardianship or conservatorship. Each account holder can perform legal actions related to the savings accoun

### Tool 5 - Retrieve products from Customer

Define a function that retrieves the products from a customer

In [29]:
def retrieve_products_of_customers(customer_id) -> pd.DataFrame:
    """Retrieve the products of a customer in the database. Customers are specified with their id. """
    return driver.execute_query(
        """
        MATCH (c:Customer)-[:HAS_PRODUCT]->(p:Product)
        WHERE c.id = $customer_id
        RETURN p.id as product_id, p.name as product_name
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        customer_id = customer_id,
        result_transformer_= lambda r: r.to_df(),
    )

In [30]:
retrieve_products_of_customers(16)

,product_id,product_name
0,ef31587f-5c96-4ab2-99e6-99f3b6fa88e8,DirectRekening Product
1,f64aae41-23d9-45d7-9310-48c7a1fd158b,RaboBusiness Banking Product
2,576fd5fe-5d27-424a-865f-56617a2955a7,Kortlopende Reis Product


### Tool 6 - Retrieve information from a product

Function retrieving all properties of a product

In [31]:
def retrieve_information_from_product(product_id) -> pd.DataFrame:
    """Retrieve the information of a product in the database. Product is specified with id."""
    result = driver.execute_query(
        """
       MATCH (p:Product{id: $product_id})
        RETURN properties(p) as props
        """,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        product_id = product_id,
        result_transformer_= lambda r: r.to_df(),
    )
    return result.iloc[0]['props']

In [32]:
retrieve_information_from_product("ef31587f-5c96-4ab2-99e6-99f3b6fa88e8")

{'id': 'ef31587f-5c96-4ab2-99e6-99f3b6fa88e8',
 'expirationDate': neo4j.time.Date(2020, 4, 1),
 'name': 'DirectRekening Product',
 'iban': 'NL85RABO1352165826'}

### Tool 7 - Retrieve Customer ID based on full name

Retrieve customer id from the database based on name

In [33]:
def retrieve_customer_id_from_database_based_on_full_name(full_name) -> pd.DataFrame:
    """Retrieve customers from the database database based on full_name. customer_id is returned."""
    
    query = """MATCH (c:Customer) WHERE c.name = $name RETURN c.id"""

    result = driver.execute_query(
        query,
        database_=DATABASE,
        routing_=RoutingControl.READ,
        name = full_name,
        result_transformer_= lambda r: r.to_df(),
    )
    return result

In [34]:
retrieve_customer_id_from_database_based_on_full_name("Lucas Van den Berg")

,c.id
0,48


## Setting up the Agent

In [35]:
llm = ChatOpenAI(model_name=LLM, temperature=0)

In [36]:
response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [37]:
tools = [
    retrieve_products,
    map_product_to_database_products,
    retrieve_document_from_product,
    answer_question_in_document,
    retrieve_products_of_customers,
    retrieve_information_from_product,
    retrieve_customer_id_from_database_based_on_full_name,
]

llm_with_tools = llm.bind_tools(tools)

## Running Agents with LangGraph

We are creating a simple [react agent](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent) using Langgraph 

In [38]:
agent_executor = create_react_agent(llm, tools)

In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

In [40]:
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='bc35a925-57b8-4840-9722-83f7c40bc960'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 264, 'total_tokens': 275, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'finish_reason': 'stop', 'logprobs': None}, id='run-3226430e-e4c8-4ac6-8233-2f425b7e8a1d-0', usage_metadata={'input_tokens': 264, 'output_tokens': 11, 'total_tokens': 275, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

#### Run some examples! 

In [41]:
def ask_to_agent(question):
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content=question)]},
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()

In [42]:
question = "What Products does Jan Kok have?"

In [43]:
ask_to_agent(question)

================================ Human Message =================================

What Products does Jan Kok have?
================================== Ai Message ==================================

To find out what products Jan Kok has, I first need to retrieve the customer ID for Jan Kok. Let me do that for you.
Tool Calls:
  retrieve_customer_id_from_database_based_on_full_name (call_Si5oYQn0T9aKjrFAhRzK8oxX)
 Call ID: call_Si5oYQn0T9aKjrFAhRzK8oxX
  Args:
    full_name: Jan Kok
================================= Tool Message =================================
Name: retrieve_customer_id_from_database_based_on_full_name

   c.id
0    16
================================== Ai Message ==================================
Tool Calls:
  retrieve_products_of_customers (call_qK5ZBA2NTzzxeB75pqKt8Yf4)
 Call ID: call_qK5ZBA2NTzzxeB75pqKt8Yf4
  Args:
    customer_id: 16
================================= Tool Message =================================
Name: retrieve_products_of_customers

            

In [44]:
question = "I got a question on my savings account, what are the rules for a joint account?"

In [45]:
ask_to_agent(question)

================================ Human Message =================================

I got a question on my savings account, what are the rules for a joint account?
================================== Ai Message ==================================
Tool Calls:
  retrieve_products (call_5ebpNXPuhWLVOZqmqZCZUUeJ)
 Call ID: call_5ebpNXPuhWLVOZqmqZCZUUeJ
  Args:
================================= Tool Message =================================
Name: retrieve_products

["SpaarRekening", "DirectRekening", "Kortlopende Reis", "BeleggersRekening", "RaboBusiness Banking"]
================================== Ai Message ==================================
Tool Calls:
  map_product_to_database_products (call_dfrVKJLnoPyGkVoDPDsGmx9K)
 Call ID: call_dfrVKJLnoPyGkVoDPDsGmx9K
  Args:
    product: savings account
================================= Tool Message =================================
Name: map_product_to_database_products

Product: SpaarRekening
================================== Ai Message ===========

In [46]:
question = "When is my travel insurance exprired? My name is Daan Visser"

In [47]:
ask_to_agent(question)

================================ Human Message =================================

When is my travel insurance exprired? My name is Daan Visser
================================== Ai Message ==================================
Tool Calls:
  retrieve_customer_id_from_database_based_on_full_name (call_5NsAoVkjTKZGsnkZBePAkPEY)
 Call ID: call_5NsAoVkjTKZGsnkZBePAkPEY
  Args:
    full_name: Daan Visser
================================= Tool Message =================================
Name: retrieve_customer_id_from_database_based_on_full_name

   c.id
0    47
================================== Ai Message ==================================
Tool Calls:
  retrieve_products_of_customers (call_nzTddoZv8cvuPt8FF3byaizW)
 Call ID: call_nzTddoZv8cvuPt8FF3byaizW
  Args:
    customer_id: 47
================================= Tool Message =================================
Name: retrieve_products_of_customers

                             product_id                  product_name
0  5494327a-a164-43c0-99b6-b

In [57]:
question = "When is my IBAN of my Saving account? My name Lucas van den Berg"

In [49]:
ask_to_agent(question)

================================ Human Message =================================

When is my IBAN of my Saving account? My name Lucas vd Berg
================================== Ai Message ==================================
Tool Calls:
  retrieve_customer_id_from_database_based_on_full_name (call_T8UVEInkJJonLz2twEQP8Y5U)
 Call ID: call_T8UVEInkJJonLz2twEQP8Y5U
  Args:
    full_name: Lucas vd Berg
================================= Tool Message =================================
Name: retrieve_customer_id_from_database_based_on_full_name

Empty DataFrame
Columns: [c.id]
Index: []
================================== Ai Message ==================================

I couldn't find any customer information for "Lucas vd Berg" in the database. Please ensure that the name is correct or provide additional details to help locate your account.


## Chatbot

Now create a chatbot with the agent providing the responses

In [50]:
def user(user_message, history):
    if history is None:
        history = []
    history.append({"role": "user", "content": user_message})
    return "", history

def get_answer(history):
    steps = []
    full_prompt = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in history])
    
    for step in agent_executor.stream(
            {"messages": [HumanMessage(content=full_prompt)]},
            stream_mode="values",
    ):
        step["messages"][-1].pretty_print()
        steps.append(step["messages"][-1].content)
    
    return steps[-1]

def bot(history):
    bot_message = get_answer(history)
    history.append({"role": "assistant", "content": ""})

    for character in bot_message:
        history[-1]["content"] += character
        time.sleep(0.01)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        label="Chatbot on a Graph",
        avatar_images=[
            "https://png.pngtree.com/png-vector/20220525/ourmid/pngtree-concept-of-facial-animal-avatar-chatbot-dog-chat-machine-illustration-vector-png-image_46652864.jpg",
            "https://d-cb.jc-cdn.com/sites/crackberry.com/files/styles/larger/public/article_images/2023/08/openai-logo.jpg"
        ],
        type="messages", 
    )
    msg = gr.Textbox(label="Message")
    clear = gr.Button("Clear")

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot], chatbot
    )

    clear.click(lambda: [], None, chatbot, queue=False)

demo.queue()
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://2d895c5cd2d39bb1fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


If you want to have the light-mode for the chatbot paste the following after the URL: /?__theme=light

### LangGraph Workflow

Within LangGraph you can also define [Agent Workflows](https://langchain-ai.github.io/langgraph/tutorials/workflows/). Below we are quickly setting up such a workflow. 

In [51]:
retrieve_product_customer_prompt = """
As an intelligent assistant, your primary objective is to find either a customer name or product from the text submitted. 
The goal is to retrieve either the product on which someone is asking a question. Or the customer name such that we can find the products from their. 
Please only return the product name that is extracted. 

Examples:
#####
User: I got a question about my savings account. 
Assistant: Product: savings account
#####
#####
User: My name is Jan Blok and I got a question. 
Assistant: Customer: Jan Blok
#####
#####
User: What is the policy on account? 
Assistant: Need more information
#####
"""

def retrieve_product_or_customer_from_text(question):
    """Retrieve either products or customers from the text given by the user"""

    response = client.beta.chat.completions.parse(
        model=LLM,
        temperature=0,
        messages=[
            {"role": "system", "content": retrieve_product_customer_prompt},
            {"role": "user", "content": question},
        ],
#        response_format=DefinitionList,
    )
    return response.choices[0].message.content 

In [52]:
class State(TypedDict):
    input: str
    question: str
    product: str
    customer: str
    document: str
    answer: str

from langgraph.types import interrupt, Command

# no-op node that should be interrupted on
def human_feedback(state: State):
    pass


def retrieve_product_customer(state):
    result = retrieve_product_or_customer_from_text(state['input'])
    if 'Customer: ' in result:
        customer = result.split('Customer: ')[1]
        print('It looks like you are the following customer: ' + customer)
        state['customer'] = customer
    elif 'Product: ' in result:
        product = result.split('Product: ')[1]
        print('It sounds like you got a question on the following product: ' + product)
        state['product'] = product
    return state

def user_feedback(state):
    print("I couldn't find information to start the flow. Could you specify a product on which you have questions or your customer name?")
    return state 
    
def map_product_to_db(state):
    result = map_product_to_database_products(state['product'])
    product = result.split('Product: ')[1]
    state['product'] = product
    print("I have found the following product in the database: " + product)
    return state

def retrieve_document(state):
    result = retrieve_document_from_product(state['product'])
    state['document'] = result
    print("I found the following document on the product: " + result)
    return state

def tools_condition(state) -> Literal["user_feedback", "map_product"]: 
    if (state.get('product') is None):
        return "user_feedback"
    else: 
        return "map_product"

def answer_question(state):
    context = perform_search_in_document(state['question'], state['document'])
    theprompt = generate_prompt(state['question'], context)
    state['answer'] = llm(theprompt.to_messages()).content
    return state 
    
# Build graph
builder = StateGraph(State)
builder.add_node("retrieve_product_customer", retrieve_product_customer)
builder.add_node("user_feedback", user_feedback)
builder.add_node("map_product", map_product_to_db)
builder.add_node("retrieve_document", retrieve_document)
builder.add_node("human_feedback", human_feedback)
builder.add_node("answer_question", answer_question)

# Logic
builder.add_edge(START, "retrieve_product_customer")
builder.add_conditional_edges("retrieve_product_customer", tools_condition)
builder.add_edge("user_feedback", END)
builder.add_edge("map_product", "retrieve_document")
builder.add_edge("retrieve_document", "human_feedback")
builder.add_edge("human_feedback", "answer_question")
builder.add_edge("answer_question", END)

memory = MemorySaver()

# Add
graph = builder.compile(checkpointer=memory, interrupt_before=["human_feedback"])

In [53]:
graph.get_graph().print_ascii()

                   +-----------+                   
                   | __start__ |                   
                   +-----------+                   
                          *                        
                          *                        
                          *                        
           +---------------------------+           
           | retrieve_product_customer |           
           +---------------------------+           
                 ...            ...                
               ..                  ...             
             ..                       ..           
   +-------------+                      ..         
   | map_product |                       .         
   +-------------+                       .         
          *                              .         
          *                              .         
          *                              .         
+-------------------+                    .         
| retrieve_d

In [54]:
initial_input = {"input" : "I got a question on my savings account, what are the rules for a shared account?"}

thread  = {"configurable": {"thread_id": "25"}}
for event in graph.stream(initial_input, thread, stream_mode="values"):
    print(event)

print(graph.get_state(thread))

user_input = input(f"Please raise your question on {graph.get_state(thread).values['product']}: ")

graph.update_state(thread, {"question": user_input}, as_node="human_feedback")

graph.get_state(thread).next

for event in graph.stream(None, thread, stream_mode="values"):
    print(event)

{'input': 'I got a question on my savings account, what are the rules for a shared account?'}
It sounds like you got a question on the following product: savings account
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'savings account'}
I have found the following product in the database: SpaarRekening
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'SpaarRekening'}
I found the following document on the product: Rabo SpaarRekening 2020.pdf
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'SpaarRekening', 'document': 'Rabo SpaarRekening 2020.pdf'}
StateSnapshot(values={'input': 'I got a question on my savings account, what are the rules for a shared account?', 'product': 'SpaarRekening', 'document': 'Rabo SpaarRekening 2020.pdf'}, next=('human_feedback',), config={'configurable': {'thread_id': '25', 'checkpoint_ns': '', 

Please raise your question on SpaarRekening:  what are the rules on a joint account?


{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'question': 'what are the rules on a joint account?', 'product': 'SpaarRekening', 'document': 'Rabo SpaarRekening 2020.pdf'}
{'input': 'I got a question on my savings account, what are the rules for a shared account?', 'question': 'what are the rules on a joint account?', 'product': 'SpaarRekening', 'document': 'Rabo SpaarRekening 2020.pdf', 'answer': 'The rules for a joint account, as outlined in the provided context, are as follows:\n\n1. **Joint Account Usage**: If the savings account is a joint account, it can only be used together by all account holders. This means that any actions related to the account must be agreed upon and executed by all account holders collectively. (Source: Rabo SpaarRekening 2020.pdf, page 8, chunk 71)\n\n2. **Communication and Notifications**: When there are multiple account holders, the bank only needs to inform one account holder, either in writing or electron

In [55]:
graph.get_state(thread).values

{'input': 'I got a question on my savings account, what are the rules for a shared account?',
 'question': 'what are the rules on a joint account?',
 'product': 'SpaarRekening',
 'document': 'Rabo SpaarRekening 2020.pdf',
 'answer': 'The rules for a joint account, as outlined in the provided context, are as follows:\n\n1. **Joint Account Usage**: If the savings account is a joint account, it can only be used together by all account holders. This means that any actions related to the account must be agreed upon and executed by all account holders collectively. (Source: Rabo SpaarRekening 2020.pdf, page 8, chunk 71)\n\n2. **Communication and Notifications**: When there are multiple account holders, the bank only needs to inform one account holder, either in writing or electronically. It is the responsibility of the informed account holder to share this information with the other account holders. All account holders are bound by the information provided to any one of them. (Source: Rabo S

Final Answer

In [56]:
print(graph.get_state(thread).values['answer'])

The rules for a joint account, as outlined in the provided context, are as follows:

1. **Joint Account Usage**: If the savings account is a joint account, it can only be used together by all account holders. This means that any actions related to the account must be agreed upon and executed by all account holders collectively. (Source: Rabo SpaarRekening 2020.pdf, page 8, chunk 71)

2. **Communication and Notifications**: When there are multiple account holders, the bank only needs to inform one account holder, either in writing or electronically. It is the responsibility of the informed account holder to share this information with the other account holders. All account holders are bound by the information provided to any one of them. (Source: Rabo SpaarRekening 2020.pdf, page 8, chunk 68)

3. **Legal Actions and Power of Attorney**: Each account holder may separately perform legal actions related to the savings account, such as granting a power of attorney, notifying a change of add